In [1]:
from googleads import adwords
import pandas as pd

In [2]:
def get_accounts(client):
    
    managed_customer_service = client.GetService('ManagedCustomerService', version='v201802')
    selector = {
                'fields': ['Name','CustomerId'],
                'predicates': [
                    {
                        'field': 'Name',
                        'operator': 'IN',
                        'values': list(df['AccountDescriptiveName'].unique())
                    }
                ]
    }
    accounts = managed_customer_service.get(selector)
    return accounts

def upload_negatives(client):
    accounts = get_accounts(client)

    try:
        for entry in accounts['entries']:
            #iterates through accounts
            account_name = entry['name']
            client.client_customer_id = entry['customerId']
            ad_group_criterion_service = client.GetService('AdGroupCriterionService', version='v201802')
            
            adgroups = list(df[df['AccountDescriptiveName'] == account_name]['AdGroupId'].unique())
            for adgroup in adgroups:
                
                terms = list(df[(df['AccountDescriptiveName'] == account_name) & (df['AdGroupId'] == adgroup)]['Query'])
                for term in terms:
                    try:
                        keyword = {
                            'xsi_type': 'NegativeAdGroupCriterion',
                            'adGroupId': adgroup,
                            'criterion': {
                                'xsi_type': 'Keyword',
                                'matchType': 'EXACT',
                                'text': term
                            }
                        }

                        operations = [
                            {
                                'operator': 'ADD',
                                'operand': keyword
                            }
                        ]

                        ad_group_criteria = ad_group_criterion_service.mutate(operations)['value']
                    except Exception,e:
                        print('Error with ' + term)
            
    except Exception,e:
        print(e)

In [3]:
if __name__ == '__main__':    # Check that this whole script is actually being run directly (as the main script) Then run the below
    df = pd.read_excel('negatives.xlsx')
    df.reset_index(inplace=True)
    adwords_client = adwords.AdWordsClient.LoadFromStorage('../googleads.yaml')
    upload_negatives(adwords_client)

No handlers could be found for logger "googleads.common"
